# Exemplo Modelo de regressão

Neste exemplo iremos desenvolver um modelo para prever os gastos com cartão de uma pessoa.

## Carregar pacotes

In [1]:
library(tidyverse)
library(magrittr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



## Carregar dados

In [2]:
dados_cartao <- read_csv( file = "/home/vm-data-science/dados/base_gastos_cartao.csv")

Parsed with column specification:
cols(
  gastos_cartao = col_integer(),
  idade = col_integer(),
  renda = col_integer(),
  impostos = col_integer(),
  segmento = col_character()
)


In [3]:
dados_cartao %>% 
    head()

gastos_cartao,idade,renda,impostos,segmento
510,35,1120,60,C
490,30,1120,60,C
470,32,1040,60,C
460,31,1200,60,C
500,36,1120,60,C
540,39,1360,120,C


## Análise exploratória

In [ ]:
dados_cartao %>% 
    summarise_if( is.numeric, mean )

In [ ]:
dados_cartao  %>% 
    count( segmento )

## Dividir amostra de treino e teste

In [18]:
# amostra de treino
dados_cartao_train <- dados_cartao %>% 
                        sample_frac(., 0.5)

In [19]:
# amostra de teste
dados_cartao_test <- setdiff( dados_cartao, dados_cartao_train )

In [20]:
print( paste('o banco de dados tem ao todo', dim(dados_cartao)[1], 'linhas e', dim(dados_cartao)[2], 'colunas') )
print( paste('a amostra de treino tem', dim(dados_cartao_train)[1], 'linhas e', dim(dados_cartao_train)[2], 'colunas') )
print( paste('a amostra de teste tem', dim(dados_cartao_test)[1], 'linhas e', dim(dados_cartao_test)[2], 'colunas') ) 

[1] "o banco de dados tem ao todo 150 linhas e 5 colunas"
[1] "a amostra de treino tem 75 linhas e 5 colunas"
[1] "a amostra de teste tem 74 linhas e 5 colunas"


## Treinamento do modelo

Conforme aprendemos na aula, vamos testar o seguinte:

- Modelo somente utilizando a variável renda
- Modelo com todas as variáveis, inclusive as categóricas
- Modelo contendo efeitos quadráticos
- Modelo contendo efeitos multiplicativos (interação)

### Modelo 1 - somente renda

In [7]:
modelo_1 <- lm( formula = gastos_cartao ~ renda, 
                data = dados_cartao_train)

In [8]:
summary( modelo_1 )


Call:
lm(formula = gastos_cartao ~ renda, data = dados_cartao_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-73.336 -28.712  -6.029  25.323 106.294 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 4.229e+02  1.143e+01   36.99   <2e-16 ***
renda       5.289e-02  3.306e-03   16.00   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 38.61 on 73 degrees of freedom
Multiple R-squared:  0.778,	Adjusted R-squared:  0.775 
F-statistic: 255.9 on 1 and 73 DF,  p-value: < 2.2e-16


### Modelo 2 - todas as variáveis

In [21]:
modelo_2 <- lm( formula = gastos_cartao ~ ., 
    data = dados_cartao_train)

In [22]:
summary( modelo_2 )


Call:
lm(formula = gastos_cartao ~ ., data = dados_cartao_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-70.497 -17.954  -2.818  21.087  75.801 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 151.34588   50.18907   3.016  0.00359 ** 
idade         3.89235    1.28902   3.020  0.00355 ** 
renda         0.09445    0.01228   7.691 7.36e-11 ***
impostos     -0.03954    0.07655  -0.517  0.60709    
segmentoB    20.14197   18.81832   1.070  0.28820    
segmentoC   108.85581   52.55590   2.071  0.04208 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 30.31 on 69 degrees of freedom
Multiple R-squared:  0.8831,	Adjusted R-squared:  0.8746 
F-statistic: 104.2 on 5 and 69 DF,  p-value: < 2.2e-16


### Modelo 3 - efeitos quadráticos (idade)

In [11]:
modelo_3 <- lm( formula = gastos_cartao ~ idade + renda + impostos + segmento + I(idade^2), 
                data = dados_cartao_train)

In [12]:
summary( modelo_3 )


Call:
lm(formula = gastos_cartao ~ idade + renda + impostos + segmento + 
    I(idade^2), data = dados_cartao_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-72.377 -22.923  -1.341  21.611  63.307 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 356.95853  137.31694   2.600   0.0114 *  
idade        -8.19127    9.18350  -0.892   0.3756    
renda         0.10167    0.01211   8.393 4.24e-12 ***
impostos     -0.15146    0.06525  -2.321   0.0233 *  
segmentoB    17.34857   16.83956   1.030   0.3066    
segmentoC    68.27372   45.96185   1.485   0.1420    
I(idade^2)    0.20662    0.14756   1.400   0.1660    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 30.68 on 68 degrees of freedom
Multiple R-squared:  0.8694,	Adjusted R-squared:  0.8579 
F-statistic: 75.47 on 6 and 68 DF,  p-value: < 2.2e-16


#### Modelo 4 - efeitos multiplicativos

In [13]:
modelo_4 <- lm( formula = gastos_cartao ~ idade + renda + impostos + segmento + 
                          I( c(segmento == 'B') * renda ) + I( c(segmento == 'C') * renda ), 
                data = dados_cartao_train )

In [14]:
summary( modelo_4 )


Call:
lm(formula = gastos_cartao ~ idade + renda + impostos + segmento + 
    I(c(segmento == "B") * renda) + I(c(segmento == "C") * renda), 
    data = dados_cartao_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-73.856 -20.230   0.083  16.658  58.034 

Coefficients:
                               Estimate Std. Error t value Pr(>|t|)    
(Intercept)                   109.11780   63.92141   1.707  0.09244 .  
idade                           4.78322    1.14403   4.181 8.61e-05 ***
renda                           0.11450    0.01415   8.091 1.65e-11 ***
impostos                       -0.16954    0.06233  -2.720  0.00831 ** 
segmentoB                      98.75967   81.90324   1.206  0.23213    
segmentoC                     285.03142   86.03211   3.313  0.00149 ** 
I(c(segmento == "B") * renda)  -0.02040    0.02077  -0.982  0.32951    
I(c(segmento == "C") * renda)  -0.15491    0.04975  -3.114  0.00272 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

R

### Teste dos modelos

Para realizar os testes dos modelos, devemos seguir pelo menos 2 passos:

1 - Realizar as previsões.

2 - Realizar os testes.

### Previsões

In [15]:
dados_cartao_test %<>% 
    mutate( gastos_cartao_pred_m1 = predict( modelo_1, .),
            gastos_cartao_pred_m2 = predict( modelo_2, .),
            gastos_cartao_pred_m3 = predict( modelo_3, .),
            gastos_cartao_pred_m4 = predict( modelo_4, .) )

### Teste

In [16]:
rmse <- function( previsto, real ) { sqrt( mean( (previsto - real)^2 ) ) }

In [17]:
dados_cartao_test %>% 
    summarise( erro_modelo_1 = rmse( gastos_cartao_pred_m1, gastos_cartao ),
               erro_modelo_2 = rmse( gastos_cartao_pred_m2, gastos_cartao ),
               erro_modelo_3 = rmse( gastos_cartao_pred_m3, gastos_cartao ),
               erro_modelo_4 = rmse( gastos_cartao_pred_m4, gastos_cartao ) )

erro_modelo_1,erro_modelo_2,erro_modelo_3,erro_modelo_4
42.67624,31.40423,31.0555,34.00008


## Qual modelo apresentou melhor desempenho na amostra de teste?